In [ ]:
# Product assistant bot for Jira

## Introduction

This Jupyter notebook demonstrates how to:
1. Connect to Jira REST API and fetch issues
2. Analyze issues using OpenAI's API
3. Add AI-generated comments back to Jira issues

### Prerequisites:
- Jira API token
- OpenAI API key
- Python packages in requirements.txt

### Configuration:
Before running this notebook, you'll need to:
1. Set up your Jira API token
2. Configure your OpenAI API token
3. Specify your Jira project key

Note: Make sure to handle your API credentials securely and never commit them to version control.

## Set JIRA credentials

In [6]:
jira_server = "https://YOUR-COMPANY.atlassian.net/"
jira_user = "USER@YOUR-COMPANY.com"
jira_api_token = "TOKEN"
jira_project_key = "KEY"

## Connect to jira and read the issues

In [7]:
from jira import JIRA

# Connect to Jira
jira_options = {"server": jira_server}
jira = JIRA(options=jira_options, basic_auth=(jira_user, jira_api_token))

# Fetch issues
issues = jira.search_issues(f"project={jira_project_key}")


## Set OpenAI credentials

In [8]:
openai_api_key = "sk-proj-KEY"

## Analyse JIRA issue and make suggestions with OpenAI

In [9]:
from openai import OpenAI

# Connect to OpenAI
openai_client = OpenAI(
    api_key=openai_api_key
)

# Analyze issues and add comments
for issue in issues:
  title = issue.fields.summary or ""
  description = issue.fields.description or ""
  prompt = f"Analyze the following description and generate a TODO list:\n{title}"

  chat_completion = openai_client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-4o-mini-2024-07-18",
  )
  todo_text = chat_completion.choices[0].message.content.strip()
  comment = f"AI-generated TODO:\n{todo_text}"
  jira.add_comment(issue, comment)